<a href="https://colab.research.google.com/github/BingHungLee/Heat_Transfer/blob/master/%5BMGGH_SYSTEM%5D_HP_1_General_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install CoolProp
from CoolProp.HumidAirProp import HAPropsSI
from CoolProp.CoolProp import PropsSI

     |████████████████████████████████| 4.0MB 2.9MB/s 


# CoolProp Python Wrapper
http://www.coolprop.org/coolprop/wrappers/Python/index.html#python

In [0]:
#[Dry Air and Water Properties]
class PROP():

  Enthalpy = 0.0        #[J/kg] 
  Desnity =0.0          #[kg/m^3]
  Viscosity = 0.0       #[Pa s]
  Heat_Capacity = 0.0   #[J/kg/K]
  Thermal_Cond = 0.0    #[W/m/K]

  def __init__(self,T,P,Fluid):
    self.Tdb = T+273.15
    self.P = P 
    self.Enthalpy = PropsSI("H", "P", P, "T", self.Tdb, Fluid)/1000
    self.Desnity = PropsSI("D", "P", P, "T", self.Tdb, Fluid)
    self.Viscosity = PropsSI("viscosity", "P", P, "T", self.Tdb, Fluid)*1000
    self.Heat_Capacity = PropsSI("Cpmass", "P", P, "T", self.Tdb, Fluid)/1000
    self.Thermal_Cond = PropsSI("conductivity", "P", P, "T", self.Tdb, Fluid)

  def Print_Prop(self):
    print('Enthalpy',self.Enthalpy, "kJ/kg")
    print('Desnity',self.Desnity, "kg/m^3")
    print("Viscosity",self.Viscosity, "mN-s/m2")
    print('Heat_Capacity',self.Heat_Capacity, "kJ/kg-C")
    print('Thermal_Cond',self.Thermal_Cond, "W/m-C")
    print()

# CoolProp Humid Air Properties
http://www.coolprop.org/fluid_properties/HumidAir.html

In [0]:
#[Moist Air Properties]
class HAP():
     
    Enthalpy = 0.0
    Humidity_Ratio = 0.0                    #kg-water/kg-da
    Specific_Volume = 0.0                   #m3/kg
    Wet_Bulb_Temperature = 0.0
    Dew_Point_Temperature = 0.0
    Partial_pressure_of_water_vapor = 0.0
     
    def __init__(self,T,P,RH):
        self.Tdb = T+273.15
        self.P = P
        self.RH = RH

        self.Enthalpy = HAPropsSI('H','T',self.Tdb,'P',self.P,'R',self.RH)/1000
        self.Humidity_Ratio = HAPropsSI("W", "T", self.Tdb, "P", self.P, "R",self.RH)
        self.Specific_Volume = HAPropsSI("V", "T", self.Tdb, "P",self.P, "R",self.RH)
        self.Wet_Bulb_Temperature = HAPropsSI("Twb", "T", self.Tdb, "P", self.P, "R",self.RH)-273.15
        self.Dew_Point_Temperature = HAPropsSI("Tdp", "T", self.Tdb, "P", self.P, "R",self.RH)-273.15
        self.Partial_pressure_of_water_vapor = HAPropsSI("P_w", "T", self.Tdb, "P", self.P, "R",self.RH)

    def Print_Prop(self):
        print('Enthalpy',self.Enthalpy, "kJ/kg")
        print('Humidity_Ratio',self.Humidity_Ratio, "kg-water/kg-da")
        print("Specific_Volume",self.Specific_Volume, "m3/kg")
        print('Wet_Bulb_Temperature',self.Wet_Bulb_Temperature, "C")
        print('Dew_Point_Temperature',self.Dew_Point_Temperature, "C")
        print('Relative_Humidity',self.RH, "__")
        print() 

# HX LMTD Calculation
* LMTD = (ΔT1 - ΔT2) / ln(ΔT1/ΔT2)
* Counter-current Flow
  * ΔT1   = T_Hot_In - T_Cold_Out
  * ΔT2   = T_Hot_Out - T_Cold_In
* Co-current Flow
  * ΔT1   = T_Hot_In - T_Cold_In
  * ΔT2   = T_Hot_Out - T_Cold_Out

In [10]:
import math
def LMTD_CouC(Thin,Thout,Tcin,Tcout):
  dT1 = Thin-Tcout
  dT2 = Thout-Tcin
  return (dT1-dT2)/math.log(dT1/dT2)

def LMTD_CoC(Thin,Thout,Tcin,Tcout):
  dT1 = Thin-Tcin
  dT2 = Thout-Tcout
  if dT2<0:
    return "NAN"
  return (dT1-dT2)/math.log(dT1/dT2)

print(LMTD_CouC(150,105,75,115))
print(LMTD_CoC(150,105,75,115))

32.4357959731544
NAN


# Nm3/hr to kg/s
* Molar mass and volume
  * Molar mass of dry air [kg/kmol] = 28.965 [md]
  * Molar mass of water vapor  [kg/kmol] = 18.016 [mw]
  * Molar volume of ideal gas at oC [Nm3/kmol] = 22.414 [vmo]
* Volume fraction
  * Volume fraction (mole ratio) of dry air [%] = [Xd]
  * Volume fraction (mole ratio) of water vapor [%] = 100 - Xd [Xw]
* Desnity
  * Density of dry air [kg/Nm3] = 28.965/22.414 = 1.29227 [dda]
  * Density of water vapor [kg/Nm3] = 18.016/22.414 = 0.80378 [dw]
  * Density of wet air [kg/Nm3] = (dda*Xd+dw*Xw)/100  [dwa]
* Flow rate
  * Flow rate of moist air [Nm3/hr] = [FRma]
  * Flow rate of dry air [kg/hr] = FRma * Xd/100* dda 
  * Flow rate of water vapor[kg/hr] = FRma * Xw/100 *dw 
  * Flow rate of moist air [kg/hr] = FRma *dwa 

In [0]:
def FRma(w,moist_air_flow_rate_in_Nm3_hr):
 return (1.29227*(1/(1+w))+0.80378(w/(1+w)))*moist_air_flow_rate_in_Nm3_hr